In [20]:
# Importar as funções necessárias
import pandas as pd
import conexao as c
import random
# Função para obter o DataFrame
def get_dataframe(nome_tabela):
    conecta = None
    try:
        #estabelece a conexão
        conecta = c.conn()
        if conecta is None:
            raise Exception("Falha ao conectar ao banco de dados.")
        
        # Executar a consulta SQL
        query = f"SELECT * FROM {nome_tabela};"
        df = pd.read_sql_query(query, conecta)
        print(f"DataFrame criado com sucesso para a tabela: {nome_tabela}")
        return df
    except Exception as e:
        print(f"Erro ao criar DataFrame para a tabela {nome_tabela}: {e}")
        return None
    finally:
        #Fecha a conexão
        if conecta:
            c.desconecta(conecta)

# Configurar opções para exibir os DataFrames
pd.set_option('display.max_columns', None)  # Exibe todas as colunas
pd.set_option('display.max_rows', 20)       # Exibe até 20 linhas
pd.set_option('display.width', 1000)        # Ajusta a largura da tabela
pd.set_option('display.colheader_justify', 'center')  # Centraliza os titulos das colunas

#transformando em dataframe
df_empresa = get_dataframe('empresa')
df_usuario = get_dataframe('usuario')

#print dos Dataframes
if df_empresa is not None:
    print("\nTabela: Empresa")
    print(df_empresa)

if df_usuario is not None:
    print("\nTabela: Usuário")
    print(df_usuario)




Conexão bem sucedida ao banco de dados: dbPrimeiroAno


C:\Users\gabrielvigna-IEG\AppData\Local\Temp\ipykernel_24276\1188075776.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conecta)


DataFrame criado com sucesso para a tabela: empresa
Conexão encerrada.
Conexão bem sucedida ao banco de dados: dbPrimeiroAno
DataFrame criado com sucesso para a tabela: usuario
Conexão encerrada.

Tabela: Empresa
   id             nome                     cnpj                 email               telefone      segmento  porte             regradenegocio                    rua          complemento         cidade         estado    cep     horario_abertura horario_encerramento
0   1  Indústria Metais Leves S.A.  12.345.678/0001-99  contato@metaisleves.com  (11) 4455-6677  Metalurgia  Médio  Foco em produção B2B e exportação.  Av. Industrial, 1500   Galpão 3   São Bernardo do Campo   SP    09870-123     07:00:00           17:00:00      

Tabela: Usuário
   id      nome            cpf       genero status        senha         fk_setor_id  fk_supervisor_id
0   1  Carlos Ferraz  111.222.333-44    M    Ativo   hash_senha_carlos       1              NaN      
1   2      Ana Souza  555.666.777-88  

In [21]:
# -------- Funções de anonimização --------
def anonimizarUsuario(df):
    try:
        if df is None or df.empty:
            raise ValueError("O DataFrame está vazio ou nulo.")

        df['nome'] = [f'Usuario_{i+1}' for i in range(len(df))]
        # CPF como número válido
        df['cpf'] =  [str(random.randint(10000000000, 99999999999)) for _ in range(len(df))]
        df['senha'] = [f'Senha_{i+1}' for i in range(len(df))]
    except Exception as e:
        print(f'Um Erro inesperado ocorreu....{e}')
        return None
    return df



# -------- Aplicando anonimização nos dados vindos do SQL --------
df_Usuario_Anonimizado = anonimizarUsuario(df_usuario)
if df_Usuario_Anonimizado is not None:
    print("\nTabela: Usuário Anonimizado")
    print(df_Usuario_Anonimizado)



Tabela: Usuário Anonimizado
   id    nome        cpf      genero status  senha    fk_setor_id  fk_supervisor_id
0   1  Usuario_1  35144248888    M    Ativo  Senha_1       1              NaN      
1   2  Usuario_2  97186994942    F    Ativo  Senha_2       1              1.0      
2   3  Usuario_3  56946865674    M    Ativo  Senha_3       2              NaN      


In [26]:
# -------- Funções de anonimização --------
def anonimizarEmpresa(df):
    try:
        if df is None or df.empty:
            raise ValueError("O DataFrame não pode ser vazio.")

        df['nome'] = [f'Empresa_{i+1}' for i in range(len(df))]
        df['cnpj'] = [f'Cnpj_{i+1}' for i in range(len(df))]
        df['email'] = [f'empresa{i+1}@email.com' for i in range(len(df))]
        df['telefone'] = [f'Telefone_{i+1}' for i in range(len(df))]
        df['rua'] = [f'Rua_Empresa_{i+1}' for i in range(len(df))]
        df['complemento'] = [f'Complemento_Empresa_{i+1}' for i in range(len(df))]
        df['cidade'] = [f'Cidade_Empresa_{i+1}' for i in range(len(df))]
        df['estado'] = [df.loc[i, 'estado'] for i in range(len(df))]
        df['cep'] = [f'Cep_{i+1}' for i in range(len(df))]

        # Seleciona apenas as colunas que existem na tabela de destino
        colunas_esperadas = ['id', 'nome', 'cnpj', 'email', 'telefone', 'rua', 'complemento', 'cidade', 'estado', 'cep']
        df = df[colunas_esperadas]

        return df

    except Exception as e:
        print(f"Um erro inesperado ocorreu... {e}")
        return None



# -------- Aplicando anonimização nos dados vindos do SQL -------


df_empresa_anonimizado = anonimizarEmpresa(df_empresa)
if df_empresa_anonimizado is not None:
    print("\nTabela: Empresa Anonimizado")
    print(df_empresa_anonimizado)


Tabela: Empresa Anonimizado
   id    nome      cnpj         email          telefone       rua            complemento             cidade      estado  cep  
0   1  Empresa_1  Cnpj_1  empresa1@email.com  Telefone_1  Rua_Empresa_1  Complemento_Empresa_1  Cidade_Empresa_1   SP    Cep_1


In [27]:
def enviar_para_banco(df, nome_tabela):
    conecta = None
    try:
        if df is None or df.empty:
            print(f"O DataFrame para {nome_tabela} está vazio.")
            return
        
        conecta = c.conn()
        if conecta is None:
            print("Falha ao conectar ao banco de dados.")
            return
        
        cursor = conecta.cursor()
        colunas = ', '.join(df.columns)
        placeholders = ', '.join(['%s'] * len(df.columns))
        query = f"INSERT INTO {nome_tabela} ({colunas}) VALUES ({placeholders})"
        
        # Inserir linha por linha
        for row in df.itertuples(index=False, name=None):
            try:
                cursor.execute(query, row)
            except Exception as e:
                print(f"Erro ao inserir linha {row}: {e}")
                conecta.rollback()  # desfaz apenas essa linha
        conecta.commit()
        print(f"Dados enviados com sucesso para a tabela {nome_tabela}.")
    
    except Exception as e:
        print(f"Erro ao enviar dados para {nome_tabela}: {e}")
        if conecta:
            conecta.rollback()
    finally:
        if conecta:
            c.desconecta(conecta)


# -------- Enviar os DataFrames anonimizados --------
enviar_para_banco(df_Usuario_Anonimizado, 'usuario_anonimizado')
enviar_para_banco(df_empresa_anonimizado, 'Empresa_anonimizada')

Conexão bem sucedida ao banco de dados: dbPrimeiroAno
Dados enviados com sucesso para a tabela usuario_anonimizado.
Conexão encerrada.
Conexão bem sucedida ao banco de dados: dbPrimeiroAno
Erro ao inserir linha (1, 'Empresa_1', 'Cnpj_1', 'empresa1@email.com', 'Telefone_1', 'Rua_Empresa_1', 'Complemento_Empresa_1', 'Cidade_Empresa_1', 'SP', 'Cep_1'): duplicate key value violates unique constraint "empresa_anonimizada_pkey"
DETAIL:  Key (id)=(1) already exists.

Dados enviados com sucesso para a tabela Empresa_anonimizada.
Conexão encerrada.
